In [1]:
#%matplotlib inline

In [2]:
from pylab import *
import matplotlib.pyplot as plt
import numpy

All miners mine big blocks
---

In [4]:
data = numpy.loadtxt('shares_latency_all.csv', skiprows=1, delimiter=',', usecols=(0,1))

In [5]:
def latency(data, i):
    return np.array(map(lambda x: x[1], filter(lambda x: x[0] == i, data))).astype(float)

In [6]:
def histogram(data, filename, caption):
    fig, ax = plt.subplots()
    ax.set_ylabel('Number')
    ax.set_xlabel('Blocks found (%)')
    n, bins, patches = plt.hist( [latency(data, 1), latency(data, 20)], 10, rwidth=1, histtype='bar', label=['1 latency', '20 latency'])
    plt.legend()
    plt.xticks(bins, map(lambda x: round(x, 2), bins))
    plt.tick_params('x', length=20, width=2, which='major', top='off', direction='inout', pad=15)
    plt.axvline(latency(data, 1).mean(), color='k', linestyle='dashed', linewidth=2)
    plt.axvline(latency(data, 20).mean(), color='k', linestyle='dashed', linewidth=2)
    plt.title('Block shares histogram',fontsize=20)
    plt.gca().set_position((.1, .3, .8, .6))
    figtext(.02, .01, caption)
    plt.savefig(filename, dpi=200)
    plt.clf()
caption = "Fully connected network of one 30% miner and seven 10% miners.\n\
1000 simulations of 6 months per condition: either all blocks have 1 latency or 20 latency.\n\
Plot shows the distribution of the fraction of blocks in the main chain that were\nfrom the big miner."
histogram(data, 'plots/histogram.png', caption)

In [7]:
def varying_latencies(data, filename, caption):
    plt.boxplot([latency(data, 1), latency(data, 5), latency(data, 10), latency(data, 20), latency(data, 30)])
    plt.ylabel('Blocks found (%)')
    plt.xlabel('Latency')
    plt.xticks([1,2,3,4, 5],('1', '5', '10', '20', '30'))
    plt.title('Block shares for varying latencies',fontsize=20)
    plt.gca().set_position((.1, .3, .8, .6))
    figtext(.02, .04, caption)
    plt.savefig(filename, dpi=200)
    plt.clf()
caption = "Fully connected network of one 30% miner and seven 10% miners.\n\
1000 simulations of 6 months per latency condition.\n\
Boxplot shows the fraction of blocks in the main chain that were from the big miner."
varying_latencies(data, 'plots/varying_latencies.png', caption)

In [28]:
data = numpy.loadtxt('shares_latency_realistic.csv', skiprows=1, delimiter=',', usecols=(0,1,2))

In [29]:
def realistic_hashrates(data, filename, caption):
    # mean normal: every miner under normal condition
    # mean latency: 20 latency condition
    meanNormal = {}
    meanLatency = {}
    for miner in data:
        d = meanNormal
        if miner[0] == 20:
            d = meanLatency
        if miner[1] in d:
            d[miner[1]] = np.append(d[miner[1]], [miner[2]])
        else: 
            d[miner[1]] = np.array([miner[2]])

    x = map(lambda hashrate: ((meanLatency[hashrate] - meanNormal[hashrate])*100/meanNormal[hashrate]).mean(), [5, 10, 20])
    std = map(lambda hashrate: ((meanLatency[hashrate] - meanNormal[hashrate].mean())*100/meanNormal[hashrate].mean()).std(), [5, 10, 20])
    fig, ax = plt.subplots()
    ax.set_ylabel('Blocks found (% change)')
    ax.set_xlabel('Hashrate (%)')
    width = 1
    for i in range(3):
        rects1 = ax.bar([i], x[i], width, color='r' if x[i] < 0 else 'g', yerr=std[i], error_kw=dict(ecolor='k', lw=1, capsize=5, capthick=1))
    plt.xticks([width/2.0,1+width/2.0,2+width/2.0], ('5', '10', '20'))
    plt.title('Change in block shares (1 vs 20 latency)',fontsize=20)
    plt.gca().set_position((.1, .3, .8, .6))
    
    figtext(.02, .06, caption)
    plt.savefig(filename, dpi=200)
    plt.clf()
caption = "Fully connected network of two 20% miners, three 10% miners and six 5% miners.\n\
1000 simulations of 6 months.\n\
Plot shows for each miner group the percentual difference between block shares in the 1\nand 20 latency condition.\
 Error bars indicate standard deviation."
realistic_hashrates(data,'plots/realistic_hashrates.png', caption)

Only big miner mines big blocks
---

In [30]:
data = numpy.loadtxt('shares_latency_all_small_miners.csv', skiprows=1, delimiter=',', usecols=(0,1))
caption = "Fully connected network of one 30% miner and seven 10% miners.\
The 10% miners \nalways create 1 latency blocks.\
 1000 simulations of 6 months per condition: \neither all of the big miner's blocks have 1 latency or 20 latency.\n\
Plot shows the distribution of the fraction of blocks in the main chain that were\nfrom the big miner."
histogram(data, 'plots/histogram_small.png', caption)
varying_latencies(data, 'plots/varying_latencies_small.png', caption)
caption = "Fully connected network of one 30% miner and seven 10% miners.\n\
 The 10% miners always create 1 latency blocks. 1000 simulations of 6 months per latency setting for the big miner.\n\
Boxplot shows the fraction of blocks in the main chain that were from the big miner."
data = numpy.loadtxt('shares_latency_realistic_small_miners.csv', skiprows=1, delimiter=',', usecols=(0,1,2))
caption = "Fully connected network of two 20% miners, three 10% miners and six 5% miners.\n\
All but the 20% miners create 1 latency blocks. \
1000 simulations of 6 months.\n\
Plot shows for each miner group the percentual difference between block shares \nwhen the 20% miners create 1 vs 20 latency blocks.\
 Error bars indicate standard deviation."
realistic_hashrates(data,'plots/realistic_hashrates_small.png', caption)

In [31]:
data = numpy.loadtxt('shares_latency_all_small_miners_40.csv', skiprows=1, delimiter=',', usecols=(0,1))
caption = "Fully connected network of one 40% miner and six 10% miners.\
The 10% miners \nalways create 1 latency blocks.\
 1000 simulations of 6 months per condition: \neither all of the big miner's blocks have 1 latency or 20 latency.\n\
Plot shows the distribution of the fraction of blocks in the main chain that were\nfrom the big miner."
histogram(data, 'plots/histogram_small_40.png', caption)
varying_latencies(data, 'plots/varying_latencies_small_40.png', caption)

In [32]:
data = numpy.loadtxt('shares_latency_all_small_miners_40_selective.csv', skiprows=1, delimiter=',', usecols=(0,1))
caption = "Fully connected network of one 40% miner and six 10% miners.\
The 10% miners \nalways create 1 latency blocks.\
 1000 simulations of 6 months per condition: \neither all of the big miner's blocks have 1 latency or 20 latency.\n\
Plot shows the distribution of the fraction of blocks in the main chain that were\nfrom the big miner."
histogram(data, 'plots/histogram_small_40_selective.png', caption)
varying_latencies(data, 'plots/varying_latencies_small_40_selective.png', caption)

Subset of Badly Connected Nodes
---

In [3]:
data = numpy.loadtxt('shares_latency_badly_connected_big_blocks.csv', skiprows=1, delimiter=',', usecols=(0,1,2))

In [38]:
def badly_connected(data, filename, caption):
    meanGoodConnection = {}
    meanBadConnection = {}
    nGoodConnection = 2
    for miner in data:
        d = meanBadConnection
        if miner[1] < nGoodConnection:
            d = meanGoodConnection
        if miner[0] in d:
            d[miner[0]] = np.append(d[miner[0]], [miner[2]])
        else: 
            d[miner[0]] = np.array([miner[2]])

    x = map(lambda d: (d[20].mean() - d[1].mean())*100/d[1].mean(), [meanGoodConnection, meanBadConnection])
    std = map(lambda d: ((d[20] - d[1])/d[1]*100).std(), [meanGoodConnection, meanBadConnection])
    fig, ax = plt.subplots()
    ax.set_ylabel('Blocks found (% change)')
    ax.set_xlabel('Connection')
    width = 1
    for i in range(2):
        rects1 = ax.bar([i], x[i], width, color='r' if x[i] < 0 else 'g', yerr=std[i], error_kw=dict(ecolor='k', lw=1, capsize=5, capthick=1))
    #rects1 = ax.bar([1], x[1], width, color='r', yerr=std[1], error_kw=dict(ecolor='k', lw=1, capsize=5, capthick=1))
    #rects1 = ax.bar([2], x[2], width, color='g', yerr=std[2], error_kw=dict(ecolor='k', lw=1, capsize=5, capthick=1))
    plt.xticks([width/2.0,1+width/2.0], ('well connected', 'badly connected'))
    plt.title('Change in block shares (1 vs 20 latency)',fontsize=20)
    plt.gca().set_position((.1, .3, .8, .6))
    
    figtext(.02, .06, caption)
    #plt.show()
    plt.savefig(filename, dpi=200)
    plt.clf()

In [47]:
caption = "Fully connected network of five 20% miners.\n\
60% of hashrate is well connected to each other but the rest is poorly connected\n(+100s latency).\
Well connected miners always produce 1 latency blocks.\n\
Plot shows for each miner groupthe percentual difference between block shares \nwhen the badly connected miners create 1 vs 20 latency blocks.\
\nError bars indicate standard deviation.100 simulations of 1 year."
badly_connected(data, 'plots/badly_connected_big_blocks.png', caption)

Sipa's Scenario
---

In [34]:
data = numpy.loadtxt('shares_latency_sipa.csv', skiprows=1, delimiter=',', usecols=(0,1,2))

In [35]:
def sipa(data, filename, caption):
    nMiners = 7
    shares = []
    for i in range(nMiners):
        shares += [{}]
    
    for miner in data:
        d = shares[int(miner[1])]
        if miner[0] in d:
            d[miner[0]] = np.append(d[miner[0]], [miner[2]])
        else: 
            d[miner[0]] = np.array([miner[2]])

    x = map(lambda d: (d[20].mean() - d[1].mean())*100/d[1].mean(), shares)
    std = map(lambda d: ((d[20] - d[1].mean())/d[1].mean()*100).std(), shares)
    fig, ax = plt.subplots()
    ax.set_ylabel('Blocks found (% change)')
    ax.set_xlabel('Miner')
    width = 1
    for i in range(7):
        rects1 = ax.bar([i], x[i], width, color='r' if x[i] < 0 else 'g', yerr=std[i], error_kw=dict(ecolor='k', lw=1, capsize=5, capthick=1))
    #rects1 = ax.bar([1], x[1], width, color='r', yerr=std[1], error_kw=dict(ecolor='k', lw=1, capsize=5, capthick=1))
    #rects1 = ax.bar([2], x[2], width, color='g', yerr=std[2], error_kw=dict(ecolor='k', lw=1, capsize=5, capthick=1))
    plt.xticks(map(lambda x: width/2.0+x, range(nMiners)), map(lambda x: str(x), range(nMiners)))
    plt.title('Change in block shares (1 vs 20 latency)',fontsize=20)
    plt.gca().set_position((.1, .3, .8, .6))
    
    figtext(.02, .03, caption)
    #plt.show()
    plt.savefig(filename, dpi=200)
    plt.clf()

In [36]:
sipa(data, 'plots/sipa.png', 
'Two groups of miners. First 35%, 2x20%, Second 4x0.05%.\n\
Groups are well-connected internally, but are only connected\n\
to each other with a poor connection (+100s latency) between miner 0 and 3.\n\
Plot shows for each miner group the percentual difference between block shares\n\
when the miners in the big group create 1 vs 20 latency blocks.\
\nError bars indicate standard deviation.100 simulations of 1 year.')

In [41]:
data = numpy.loadtxt('shares_latency_sipa_traitor.csv', skiprows=1, delimiter=',', usecols=(0,1,2))
sipa(data, 'plots/sipa_traitor.png', 
'Two groups of miners. First 35%, 2x20%, Second 4x0.05%.Groups are well-connected\n\
internally, but are only connected to each other with a poor connection (+100s latency)\n\
between miner 0 and 3 and between 1 and 4.Plot shows for each miner group the\n\
percentual difference between block shares when the miners in the big group\n\
create 1 vs 20 latency blocks.Error bars indicate standard deviation.\n\
100 simulations of 1 year.')